# The real

This notebook is to serve as a full compareson of model using better feature extraction.  We will extract surnames, an interaction between age and fare, and the total family size and run  the set of features on different models and compare the difference.  We will compare decision trees and Adaboost with decision tree stumps since they performed better in our pocs.

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
import features
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_raw = pd.read_csv('../data/train.csv', header=0)
test_raw = pd.read_csv('../data/test.csv', header=0)

Confirm that the test set does not have any missing data we have to deal with.

In [11]:
len(test_raw) - test_raw.count()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [18]:
train_raw['is_train'] = 1
test_raw['is_train'] = 0
data_raw = pd.concat([train_raw, test_raw])

# Feature engineering

We will create a surname feature and a family size feature and then run it all through the preprocessing script. After will will add an interaction term between fare and age.  This need to happen after the script due to missing values.

In [26]:
data_raw['Fam_size'] = data_raw['SibSp'] + data_raw['Parch'] + 1

In [14]:
data_raw['Name'].str.extract('(,\s)(.*\.)', expand=False)[1].value_counts()

Mr.                          517
Miss.                        182
Mrs.                         124
Master.                       40
Dr.                            7
Rev.                           6
Major.                         2
Col.                           2
Mlle.                          2
Ms.                            1
Don.                           1
Capt.                          1
Sir.                           1
Lady.                          1
the Countess.                  1
Mrs. Martin (Elizabeth L.      1
Jonkheer.                      1
Mme.                           1
Name: 1, dtype: int64

In [24]:
titles = ['Mr.', 'Miss.', 'Mrs.', 'Master.']
data_raw['Surname'] = data_raw['Name'].str.extract('(,\s)(.*\.)', expand=False)[1]
data_raw.loc[~data_raw['Surname'].isin(titles),'Surname'] = 'Rare'

In [35]:
categorical_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Fam_size', 'Surname']
numerical_features = ['Age', 'Fare']
target = 'Survived'
data_clean, y = features.decision_tree_preprocessing(data_raw, 
                                                  target,
                                                  categorical_features=categorical_features, 
                                                  numerical_features=numerical_features, 
                                                  drop_na_columns=['Embarked'])

In [40]:
data_clean['AgeXFare'] = data_clean['Age'] * data_clean['Fare']
X = data_clean

# Building the models

**TODO** : use a grid search to tune the hyper parameters.

In [63]:
model_ada = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=2))
model_tree = tree.DecisionTreeClassifier(max_depth=5)

In [64]:
scores_ada = cross_val_score(model_ada, X, y, cv=10)
scores_tree = cross_val_score(model_tree, X, y, cv=10)

In [65]:
print("AdaBoost accuracy: %0.2f (+/- %0.2f)" % (scores_ada.mean(), scores_ada.std() * 2))
print("Decision tree accuracy: %0.2f (+/- %0.2f)" % (scores_tree.mean(), scores_tree.std() * 2))

AdaBoost accuracy: 0.82 (+/- 0.07)
Decision tree accuracy: 0.81 (+/- 0.07)


# Prediction